Wed Jun 25 18:33:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.36                 Driver Version: 566.36         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   38C    P3             11W /   30W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

2025-03-27 22:23:50.499 
  command:

    streamlit run C:\Users\Administrador\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-03-27 22:23:50.499 Session state does not function when running a script without `streamlit run`


Textarea(value='', description='Prompt:', layout=Layout(height='100px', width='100%'), placeholder='Escribe tu…

Button(button_style='primary', description='Generar Texto', style=ButtonStyle())

Output()

(Deprecated) Installing extensions with the jupyter labextension install command is now deprecated and will be removed in a future major version of JupyterLab.

Users should manage prebuilt extensions with package managers like pip and conda, and extension authors are encouraged to distribute their extensions as prebuilt packages 
Building jupyterlab assets (production, minimized)


SyntaxError: unterminated f-string literal (detected at line 82) (3264781357.py, line 82)

SyntaxError: closing parenthesis ']' does not match opening parenthesis '{' on line 100 (2001230850.py, line 107)

In [30]:
import tkinter as tk
from tkinter import ttk, simpledialog, messagebox
from PIL import Image, ImageTk
import requests
from io import BytesIO
import os
from urllib.parse import urljoin

# Base de la URL de tu API (sin endpoint)
BASE_ENDPOINT = "https://5d26-193-147-26-250.ngrok-free.app"

# Configuración principal de la ventana
root = tk.Tk()
root.title("Visor de Imágenes y Respuesta Janus")
root.geometry("700x900")
root.configure(bg="#1e1e1e")

# Etiqueta de estado
status_label = tk.Label(root, text="", fg="white", bg="#1e1e1e", font=("Arial", 12))
status_label.pack(pady=5)

# Contenedor con scrollbar
tk_container = tk.Frame(root)
tk_container.pack(fill="both", expand=True, padx=10, pady=10)
canvas = tk.Canvas(tk_container, bg="#1e1e1e")
scrollbar = ttk.Scrollbar(tk_container, orient="vertical", command=canvas.yview)
scrollable_frame = tk.Frame(canvas, bg="#1e1e1e")
scrollable_frame.bind(
    "<Configure>",
    lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
)
canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
canvas.configure(yscrollcommand=scrollbar.set)
canvas.pack(side="left", fill="both", expand=True)
scrollbar.pack(side="right", fill="y")

# ----------- Función Principal -----------
def ejecutar_pipeline(endpoint_path, prompt_data, question, numbers=None):
    status_label.config(text="⌛ Generando imágenes y esperando respuesta...", fg="white")
    root.update()
    API_URL = BASE_ENDPOINT + endpoint_path
    payload = {"prompts": prompt_data, "question": question}
    if numbers is not None:
        payload["numbers"] = numbers
    try:
        resp = requests.post(API_URL, json=payload)
        resp.raise_for_status()
        data = resp.json()
        images = data.get("generated_images", [])
        answers = data.get("answers", [])
        for w in scrollable_frame.winfo_children(): w.destroy()
        for idx, img_path in enumerate(images):
            img_url = img_path if img_path.startswith("http") else urljoin(BASE_ENDPOINT, f"/files/{os.path.basename(img_path)}")
            try:
                img_data = requests.get(img_url).content
                img = Image.open(BytesIO(img_data)).resize((256, 256))
                img_tk = ImageTk.PhotoImage(img)
                lbl_img = tk.Label(scrollable_frame, image=img_tk, bg="#2d2d2d")
                lbl_img.image = img_tk
                lbl_img.pack(pady=5)
            except Exception as e:
                tk.Label(scrollable_frame, text=f"Error cargando imagen: {e}", fg="red", bg="#1e1e1e").pack(pady=5)
            answer_text = answers[idx] if idx < len(answers) else "No response"
            tk.Label(scrollable_frame, text=f"🧠 Respuesta {idx+1}:\n{answer_text}", fg="lightgreen", bg="#1e1e1e", font=("Arial",12), wraplength=600, justify="left").pack(pady=(0,15))
        status_label.config(text="✅ Proceso completado", fg="lightgreen")
    except Exception as e:
        status_label.config(text=f"❌ Error: {e}", fg="red")

# ----------- Handler para el primer botón -----------
def handler_primer_boton():
    # Pedir 3 prompts manuales
    prompts = []
    for i in range(1, 4):
        p = simpledialog.askstring("Prompt Manual", f"Introduce el prompt {i}:")
        if p is None:
            return
        prompts.append(p)
    # Pedir lista de números
    nums_str = simpledialog.askstring("Lista de Números", "Introduce una lista de números separados por coma, ej: 1,2,3,4:")
    if nums_str is None:
        return
    try:
        numbers = [int(x.strip()) for x in nums_str.split(",") if x.strip()]
        if not numbers:
            raise ValueError
    except ValueError:
        messagebox.showerror("Error", "Debes introducir al menos un número válido separado por coma.")
        return
    # Ejecutar pipeline con prompts y lista de números
    ejecutar_pipeline(
        endpoint_path="/generate_full",
        prompt_data=prompts,
        question="tell me a story about the pic",
        numbers=numbers
    )

def handler_segundo_boton():
    # Pedir 3 prompts manuales
    prompts = []
    for i in range(1, 4):
        p = simpledialog.askstring("Prompt Manual", f"Introduce el prompt {i}:")
        if p is None:
            return
        prompts.append(p)
    # Pedir lista de números
    nums_str = simpledialog.askstring("Lista de Números", "Introduce una lista de números separados por coma, ej: 1,2,3,4:")
    if nums_str is None:
        return
    try:
        numbers = [int(x.strip()) for x in nums_str.split(",") if x.strip()]
        if not numbers:
            raise ValueError
    except ValueError:
        messagebox.showerror("Error", "Debes introducir al menos un número válido separado por coma.")
        return
    # Ejecutar pipeline con prompts y lista de números
    ejecutar_pipeline(
        endpoint_path="/generate_custom",
        prompt_data=prompts,
        question="tell me a story about the pic",
        numbers=numbers
    )

# ----------- Configuración de botones -----------
BUTTON_CONFIGS = [
    {"text": " Fusion XL+FKConform", "command": handler_primer_boton},
    {"text": " Fusion FK+Conform+LCM", "command": handler_segundo_boton},
    {"text": "Lovecraft", "endpoint": "/generate_and_answer", "prompt_data": ["an image of a scene from lovecraft, first scene", "an image of a scene from lovecraft, mid scene", "an image of a scene from lovecraft, there is a monster"], "question": "tell me a story about this pic "},
    {"text": " Epic Fantasy", "endpoint": "/generate_and_answer", "prompt_data": ["An epic fantasy landscape at dawn, a young warrior standing at the edge of an ancient forest, a glowing map in hand, mist rising from the trees, distant mountains under a golden sky, magical birds flying overhead, sense of adventure and mystery, ultra-detailed, cinematic light, fantasy concept art, ArtStation, 8k", "A vast enchanted battlefield under a stormy sky, the warrior now in ornate armor leading a small group of companions, dark creatures emerging from a portal, lightning strikes in the distance, floating ruins and magical runes glowing red, tension and danger, epic lighting, ultra-detailed fantasy art, dark fantasy tone, 8k, high realism", "A dramatic mountaintop at sunset, the warrior standing victorious with a glowing sword, wounded but proud, the broken portal behind him, golden light illuminating the battlefield, remnants of the magical storm clearing, companions celebrating or mourning, epic closure, cinematic composition, highly detailed, fantasy concept art, 8k resolution"], "question": "tell me a story about this pic "},
    {"text": " Folk Story", "endpoint": "/generate_and_answer", "prompt_data": ["A cheerful countryside in spring, a curious child walking through a colorful meadow, tiny fairies fluttering among glowing mushrooms, a cozy village in the background, bright daylight, playful animals watching, a sense of wonder and innocence, whimsical style, storybook illustration, highly detailed, soft colors, 8k, fantasy folklore art", "A magical forest filled with talking animals and glowing plants, the child exploring a winding path under a canopy of golden leaves, mysterious lights leading the way, a wise old owl perched on a twisted branch, soft magical glow, friendly forest creatures surrounding the child, storybook fantasy, vibrant colors, richly detailed, folklore art style, 8k", "A peaceful clearing under the stars, the child receiving a glowing crystal from a fairy queen, gentle moonlight illuminating the scene, the enchanted forest calm and sparkling with magic, forest creatures celebrating, warm and joyful mood, fairy-tale illustration, charming and colorful, detailed storybook style, 8k resolution"], "question": "tell me a story about this pic "}
]
    

btn_frame = tk.Frame(root, bg="#1e1e1e")
btn_frame.pack(pady=15)
for idx, cfg in enumerate(BUTTON_CONFIGS):
    cmd = cfg.get("command") if cfg.get("command") else lambda c=cfg: ejecutar_pipeline(c["endpoint"], c["prompt_data"], c["question"])
    btn = tk.Button(btn_frame, text=cfg["text"], command=cmd, bg="#444", fg="white", font=("Arial",12), width=25)
    btn.grid(row=idx//2, column=idx%2, padx=10, pady=5)
root.mainloop()



In [32]:
dockerignore = """
__pycache__/
*.pyc
.ipynb_checkpoints/
data/
outputs/
"""
with open('.dockerignore', 'w') as f:
    f.write(dockerignore.strip())
print(".dockerignore creado.")



.dockerignore creado.


In [33]:
dockerfile = """
# 1) Base
FROM python:3.10-slim

# 2) Directorio de trabajo
WORKDIR /app

# 3) Copiar dependencias y .dockerignore
COPY requirements.txt ./
RUN pip install --no-cache-dir -r requirements.txt

# 4) Copiar tu notebook, scripts y demás
COPY . .

# 5) Exponer puerto Jupyter
EXPOSE 8888

# 6) Comando por defecto
CMD ["jupyter", "lab", "--ip=0.0.0.0", "--allow-root", "--no-browser"]
"""
with open('Dockerfile', 'w') as f:
    f.write(dockerfile.strip())
print("Dockerfile creado.")


Dockerfile creado.


In [34]:
# Le damos un nombre a la imagen
imagen = "mi_pipeline_notebook:latest"

# Lanza el build de Docker
!docker build -t {imagen} .


"docker" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
